In [1]:
import pandas as pd
import csv
import json

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
links = pd.read_csv('links.csv')

new_movies = movies.drop(['genres'],axis=1)
# ratings = pd.merge(movies,ratings).drop(['genres','timestamp'], axis = 1)
ratings = pd.merge(movies[["movieId","title"]], ratings[["userId", "movieId", "rating"]], on="movieId", how="left")
# links[["movieId", "tmdbId"]],
ratings = pd.merge(ratings, links[["movieId", "tmdbId"]], on="movieId", how="left")
ratings.head()


,movieId,title,userId,rating,tmdbId
0,1,Toy Story (1995),1.0,4.0,862.0
1,1,Toy Story (1995),5.0,4.0,862.0
2,1,Toy Story (1995),7.0,4.5,862.0
3,1,Toy Story (1995),15.0,2.5,862.0
4,1,Toy Story (1995),17.0,4.5,862.0


In [3]:
user_ratings=ratings.pivot_table(index=['userId'],columns=['tmdbId'],values='rating')
user_ratings.head()

tmdbId,2.0,5.0,6.0,11.0,12.0,13.0,14.0,15.0,16.0,18.0,...,490003.0,490928.0,494368.0,497520.0,500475.0,500609.0,502616.0,502892.0,503475.0,525662.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,5.0,NaN,4.0,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN
4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user_ratings = user_ratings.dropna(thresh=10, axis=1).fillna(0)
user_ratings.head()

tmdbId,5.0,11.0,12.0,13.0,14.0,15.0,16.0,18.0,19.0,22.0,...,330459.0,333371.0,335984.0,354912.0,370646.0,374720.0,381284.0,383498.0,419430.0,503475.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,5.0,0.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
4.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)
print(item_similarity_df.shape)

(2269, 2269)


In [6]:
def get_similar_movies(tmdb_Id, user_rating):
    similar_score = item_similarity_df[tmdb_Id]*(float(user_rating)-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [7]:
final = []
input = {
    "13.0": 5,
    "18.0": 3,
    "5.0": 1
  }
for i in input:
  k = [i, input[i]]
  final.append(k)
    
final

[['13.0', 5], ['18.0', 3], ['5.0', 1]]

In [8]:
action_lover = final

similar_movies = pd.DataFrame()

for list in action_lover:
    tmdb_id = list[0]
    rating = list[1]
    similar_movies = similar_movies.append(get_similar_movies(float(tmdb_id),rating), ignore_index=True)
    

# similar_movies = similar_movies.to_frame()
final_output = similar_movies.sum().sort_values(ascending=False)
# final_movie = pd.DataFrame({'tmdbId':final_movie.index, 'similar':final_movie.values})
# tmdbIds = final_movie.loc[:,'tmdbId']
# movie_ids = []
# imdbIds = []
# for title in titles:
#     movie_id = new_movies.loc[new_movies['title'] == title, 'movieId'].iloc[0]
#     movie_ids.append(movie_id)
#     imdbIds.append(links.loc[links['movieId'] == movie_id, 'tmdbId'].iloc[0])
    
# final_movie['movieId'] = movie_ids
# final_movie['tmdbId'] = imdbIds
#type(final_output)
json_output = final_output.to_json()
final=json.dumps(json_output)

json_output


C:\Users\KS401\AppData\Local\Temp\ipykernel_3124\4217848499.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar_movies(float(tmdb_id),rating), ignore_index=True)
C:\Users\KS401\AppData\Local\Temp\ipykernel_3124\4217848499.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar_movies(float(tmdb_id),rating), ignore_index=True)
C:\Users\KS401\AppData\Local\Temp\ipykernel_3124\4217848499.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar_movies(float(tmdb_id),rating), ignore_index=True)


'{"13.0":2.5159840588,"329.0":1.125161051,"788.0":1.0801180937,"568.0":1.0196929395,"278.0":0.966413675,"114.0":0.9656211002,"8587.0":0.9217395991,"197.0":0.9102911479,"607.0":0.9040715997,"280.0":0.8799666588,"680.0":0.8767045045,"1637.0":0.8457474769,"812.0":0.8358822752,"745.0":0.8357287161,"497.0":0.8234397532,"858.0":0.8194429401,"5503.0":0.8113959092,"18.0":0.8109091567,"424.0":0.8007617177,"137.0":0.8001756022,"771.0":0.7972581332,"105.0":0.7916976879,"251.0":0.7840945561,"807.0":0.7837669181,"489.0":0.7761453501,"9331.0":0.7741872037,"12.0":0.762784265,"36955.0":0.7551684131,"581.0":0.7449031443,"857.0":0.7368821465,"274.0":0.7260048537,"640.0":0.7245680404,"710.0":0.7212623487,"854.0":0.7179450816,"603.0":0.7140480267,"196.0":0.7050255519,"9377.0":0.7010038615,"861.0":0.6951508308,"597.0":0.692382366,"2164.0":0.688929226,"9800.0":0.6764989907,"712.0":0.6707557356,"9359.0":0.670271894,"8467.0":0.6653583644,"550.0":0.6629965119,"10020.0":0.662426157,"98.0":0.6581638349,"11566.0"